In [1]:
import sys
import os
import numpy as np
import pandas as pd
import random
import datetime
import time
from argparse import ArgumentParser
from skimage import color

np.random.seed(88)
random.seed = 88

prj_root = os.path.join(os.getcwd(), os.pardir)
sys.path.append(prj_root)
import preprocess.dyeprocess as dyeprocess
from util.dfutil import *
from core.validator import validate
from core.dyeselector import *
from core.inverse_decoder import *
from util.pickle_util import pm
from util.config_util import cm

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
decoder_input_columns = pm.decoder_input_columns

""" global vars """
DATA_FILE_ALL = os.path.join(prj_root, cm['general']['DataFileAll'])
DATA_FILE_SINGLE = os.path.join(prj_root, cm['general']['DataFileSingle'])
DECODER_PATH = os.path.join(prj_root, cm['general']['DecoderPath'])
OUTPUT_DIR = os.path.join(prj_root, cm['general']['OutputDir'])
con_dim = len(decoder_input_columns)
clothe_type_count = int(cm['general']['ClotheTypeCount'])

# for dye selector
history_num_limit = 1
max_collection_count = 1

In [3]:
DECODER_PATH

'C:\\Users\\Administrator\\Desktop\\Team31-20180925T025046Z-001\\Team31\\Everest_01_IntelligentColor_team31\\util\\..\\resource/keras_model/keras_decoder_model'

In [3]:
# loading data for selecting dyes
df_all = pd.read_csv(DATA_FILE_ALL)
df_all = df_all[df_all['abort'] != 1]
df_all = df_all[df_all['L'].notnull()]

df_single = pd.read_csv(DATA_FILE_SINGLE)
df_single = df_single[df_single['abort'] != 1]

In [5]:
input_file=os.path.join(prj_root, 'data', 'match_1011.txt')
df = pd.read_csv(input_file)
df

,LAB,400nm,410nm,420nm,430nm,440nm,450nm,460nm,470nm,480nm,...,670nm,680nm,690nm,700nm,L,a,b,C,H,布號
0,18-1250TCX,4.29,3.63,3.51,3.49,3.51,3.67,3.90,4.17,4.50,...,45.54,54.12,58.75,65.60,42.79,30.17,33.63,45.18,48.10,FVF2666
1,17-1022TCX,14.21,13.37,12.91,12.66,12.68,13.07,13.98,15.02,16.34,...,44.40,53.09,57.76,64.69,56.10,3.95,20.40,20.78,79.03,FVF2666
2,14-4320TCX,62.03,64.21,66.13,67.52,67.93,68.23,68.43,68.15,67.12,...,33.16,41.43,48.00,58.87,73.48,-13.33,-21.06,24.92,237.68,FVF2666
3,13-0650TCX,5.88,4.59,4.09,4.04,4.20,4.94,6.69,9.71,16.25,...,40.62,36.68,41.42,52.28,82.55,-11.64,81.76,82.58,98.10,FVF2666


In [7]:
inverse_decoder = InverseDecoder(DECODER_PATH, 100)
inverse_decoder.build_graph()

In [8]:
dye_selector = DyeSelector(history_num_limit=history_num_limit,
                           max_collection_count=max_collection_count)

In [9]:
preds = []
with tf.Session() as sess:
    inverse_decoder.init_model(sess)
    for row in df.loc[:, ['L','a','b','C','H', '布號']].values:
        print('row', row)
        lab = row[0:3]
        possible_collections = dye_selector.get_possible_collections(df_all, df_single, row[0:-1])
        print('possible_collections', possible_collections)
        clothe = row[-1]
        print('clothe', clothe)
        pred = inverse_decoder.predict_concentrations(possible_collections, clothe, lab)
        pred[(pred < 5e-3) & (pred != 0)] = 1e-2
        pred[pred > 3] = 3
        preds.append(pred[0])

preds = np.array(preds)

INFO:tensorflow:Restoring parameters from C:\Users\ADMINI~1\AppData\Local\Temp\2\tmpa_p_ecc7
row [42.79 30.17 33.63 45.18 48.1 'FVF2666']
possible_collections [('264-1', 'F2I', 'F4U-7')]
clothe FVF2666
('264-1', 'F2I', 'F4U-7')
epoch_100, loss:0.68488258
epoch_200, loss:0.17930737
epoch_300, loss:0.04062428
epoch_400, loss:0.02886223
epoch_500, loss:0.02794651
epoch_600, loss:0.02779328
epoch_700, loss:0.02775411
epoch_800, loss:0.02773661
epoch_900, loss:0.02772090
epoch_1000, loss:0.02770640
epoch_1100, loss:0.02769267
epoch_1200, loss:0.02768110
epoch_1300, loss:0.02766911
epoch_1400, loss:0.02765976
epoch_1500, loss:0.02765120
epoch_1600, loss:0.02764370
epoch_1700, loss:0.02763737
epoch_1800, loss:0.02763236
epoch_1900, loss:0.02762884
epoch_2000, loss:0.02762455
row [56.1 3.95 20.4 20.78 79.03 'FVF2666']
possible_collections [('228-1', '244-4', '264-1')]
clothe FVF2666
('228-1', '244-4', '264-1')
epoch_100, loss:0.45615831
epoch_200, loss:0.04877302
epoch_300, loss:0.00288879
epo

In [10]:
sess.close()

In [11]:
losses = inverse_decoder.lab_loss(preds, df.loc[:, ['L','a','b']].values)
losses

array([ 5.08555654,  1.32083891, 14.76459076,  4.04562744])

In [12]:
df_output = process_output_csv(preds, losses, decoder_input_columns, clothe, clothe_type_count, False)
df_output = pd.concat([df.loc[:,'LAB'], df_output], axis=1)
df_output['布號'] = df.loc[:, ['布號']]
df_output

,LAB,染料_1,濃度_1,染料_2,濃度_2,染料_3,濃度_3,染料_4,濃度_4,布號,delta_lab
0,18-1250TCX,264-1,0.0092,F2I,2.7649,F4U-7,0.2074,NaN,NaN,FVF2666,5.0856
1,17-1022TCX,228-1,0.5236,244-4,0.0068,264-1,0.0892,NaN,NaN,FVF2666,1.3208
2,14-4320TCX,227-1,0.0100,261-8,0.1562,264-1,0.0191,NaN,NaN,FVF2666,14.7646
3,13-0650TCX,227-1,0.1202,322-6,0.6958,NaN,NaN,NaN,NaN,FVF2666,4.0456


In [13]:
output_path = os.path.join(prj_root, 'output', '盲測1012.csv')
df_output.to_csv(output_path, index=False)